# Data cleaning + feature engineering on full data (originally done on an EC2, redone locally)

## Imports

In [1]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [ ]:
os.getcwd()

In [ ]:
# info_file = '/home/ec2-user/film-data/raw-data/info.csv'
# credits_file = '/home/ec2-user/film-data/raw-data/credits.csv'
# releases_file = '/home/ec2-user/film-data/raw-data/releases.csv'

In [2]:
info_file = '/Users/ryanrappa/Downloads/raw_data/info.csv'
credits_file = '/Users/ryanrappa/Downloads/raw_data/credits.csv'
releases_file = '/Users/ryanrappa/Downloads/raw_data/releases.csv'

In [3]:
#info table contains basic film details like genre, budget, revenue, language, runtime, etc.
info_df = pd.read_csv(info_file)

#credits table contains cast and crew
credits_df = pd.read_csv(credits_file)

#releases table contains info on which countries each film was released in
releases_df = pd.read_csv(releases_file)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (0,3,6,11,16,17,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Cleaning + new features

#### Because of the way data was collected, last row of each table has the headers. Fixing that:

In [4]:
info_df.columns = info_df.iloc[-1:, :].squeeze()
info_df = info_df.iloc[:-1, :]

In [5]:
credits_df.columns = credits_df.iloc[-1:, :].squeeze()
credits_df = credits_df.iloc[:-1, :]

In [6]:
releases_df.columns = releases_df.iloc[-1:, :].squeeze()
releases_df = releases_df.iloc[:-1, :]

### Cleaning info table

#### Getting each film's genre:

In [7]:
def get_first_genre(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    (str): name of the first genre from the list of dicts
    '''
    lst_of_dicts = eval(lod_str)
    if len(lst_of_dicts) > 0:
        for i in range(len(lst_of_dicts)):
            return lst_of_dicts[i]['name']
    else:
        pass

In [8]:
info_df['genre'] = info_df['genres'].apply(get_first_genre)  #func in cell above

#### Dropping unnecessary rows and columns from info table:

In [9]:
#making sure relevant numerical columns are int/float type
info_df = info_df.astype({"adult": str, 
                          "budget": float, 
                          "id": int, 
                          "revenue": float, 
                          "original_language": str, 
                          "runtime": float, 
                          "genre": str})

In [10]:
#filtering for only non-adult films,
# with revenue & budget > 0,
# original language english,
# runtime >= 80 minutes,
# and genre is not documentary or tv movie.

info_df = info_df[((info_df['adult'] == 'False') | (info_df['adult'] == False)) & \
        (info_df['budget'] > 0) & \
        (info_df['revenue'] > 0) & \
        (info_df['original_language'] == 'en') & \
        (info_df['runtime'] >= 80.0) & \
        (info_df['genre'] != 'Documentary') & (info_df['genre'] != 'TV Movie')]

In [11]:
#dropping columns from info_df that won't be needed for EDA
drop_cols = ['adult', 'backdrop_path', 'belongs_to_collection', 
             'genres', 'homepage', 'imdb_id', 'original_language', 'original_title', 
             'overview', 'popularity', 'poster_path', 'production_companies', 
             'production_countries', 'spoken_languages', 'status', 'tagline', 'video', 
             'vote_average', 'vote_count']

info_df = info_df.drop(drop_cols, axis=1)

### a few more cols to be dropped later for modelling, after EDA, feat. engineering, & joins: 
### ['id', 'release_date', 'runtime' (possibly), 'title', 'genre' (possibly)]

#### Creating columns for (a) amount of profit or loss (revenue minus budget) and (b) whether each movie profited (binary, whether revenue > budget):

In [12]:
info_df['profit'] = info_df['revenue'] - info_df['budget']

In [13]:
info_df['made_money'] = info_df['profit'] > 0
info_df['made_money'] = info_df['made_money'].astype('int')

In [14]:
#converting release date column from object type to datetime type
info_df['release_date'] = pd.to_datetime(info_df['release_date'])

### Joining the tables

#### Note: when the keys to join on are pd columns, not indices, better to use merge method

In [15]:
#making sure id column is int so dataframes can be merged
credits_df = credits_df.astype({"id": int})

In [16]:
#making sure id column is int so dataframes can be merged
releases_df = releases_df.astype({"id": int})

In [17]:
step1 = info_df.merge(credits_df, on='id', how='inner')

In [18]:
df = step1.merge(releases_df, on='id', how='inner')

### Cleaning credits columns

#### Appending columns with top billed actors to the table:

In [21]:
def get_top_cast(df, lod_col, billings):
    '''
    INPUT
    df: pandas dataframe w/ stringified lists of dictionaries containing cast
    lod_col (str): name of col w/ stringified lists of dictionaries
    billings (int): number of cast members to extract from each lod
    --------
    OUTPUT
    original df w/ new columns having actor names corresponding to each billing 
    number in this function (lower billing number = more prominent
    role; 0 = star of the movie, 1 = costar, etc.)
    '''
    for c in range(billings):
        billing = c
        for i in range(len(df)):
            lst_of_dicts = eval(df.at[i, lod_col]) #lod's are strings at first, hence eval
            if len(lst_of_dicts) > billing:
                dict_of_interest = lst_of_dicts[billing]
                actor = dict_of_interest['name']
                df.at[i, 'cast_'+str(billing)] = actor
            else:
                continue
    return df

In [22]:
df = get_top_cast(df, 'cast', 5)  #func in cell above

#### Appending columns with director and screenwriters to the table:

In [23]:
def get_director(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    (str): name of the first director from the list of dicts
    '''
    lst_of_dicts = eval(lod_str)
    if len(lst_of_dicts) > 0:
        for i in range(len(lst_of_dicts)):
            if lst_of_dicts[i]['job'] == 'Director':
                return lst_of_dicts[i]['name']
    else:
        pass

In [24]:
df['director'] = df['crew'].apply(get_director)  #func in cell above

In [25]:
def get_first_writer(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    (str): name of the first screenwriter from the list of dicts
    '''
    lst_of_dicts = eval(lod_str)
    if len(lst_of_dicts) > 0:
        for i in range(len(lst_of_dicts)):
            if lst_of_dicts[i]['job'] == 'Screenplay' or \
            lst_of_dicts[i]['job'] == 'Writer' or \
            lst_of_dicts[i]['job'] == 'Author':
                return lst_of_dicts[i]['name']
    else:
        pass

In [26]:
df['writer'] = df['crew'].apply(get_first_writer)  #func in cell above

In [27]:
#decided not to get second writer becuse relatively few films have one in the dataset

In [28]:
creds_drop_cols = ['cast', 'crew']
df = df.drop(creds_drop_cols, axis=1)

#### Dropping rows without at least 5 actors, or missing director/writer/release:

In [30]:
df = df.dropna(subset=['cast_4', 'director', 'writer', 'release_date'])

### Cleaning releases column

#### Getting number of countries each film was released in and dropping nulls:

In [32]:
def get_releases_count(lod_str):
    '''
    Function for use with pandas apply method.
    --------
    INPUT
    lod_str (str): a list of dictionaries in the form of a string
    --------
    OUTPUT
    (int): number of countries/territories the film was released in
    '''
    lst_of_dicts = eval(lod_str)
    if len(lst_of_dicts) > 0:
        return len(lst_of_dicts)
    else:
        pass

In [33]:
df['releases'] = df['countries'].apply(get_releases_count)  #func in cell above

In [34]:
df = df.drop('countries', axis=1)

### Checking for outliers/faulty data

In [35]:
pd.set_option('float_format', '{:f}'.format)  #so that floats will print w/o scientific notat.

In [36]:
df.describe(); #checking for outliers/faulty data

In [37]:
df = df[df['budget'] > 10000] #dropping microbudget films and faulty data

In [38]:
df = df[df['revenue'] > 10000] #dropping microbudget films and faulty data

In [39]:
df.describe(); #much better

In [40]:
# resetting index
df = df.reset_index().drop('index', axis=1)

### Getting cast & crew lifetime avg revenue, avg profits, # of films, # of profitable films

In [42]:
def get_cast_metrics(df, cast_cols):
    '''
    Iterates over columns of cast members
    and generates new columns with the avg
    revenue, avg profit, total number of movies, and total number of
    profitable movies that cast collectively has been in
    up until the date of release in each row.
    --------
    INPUT
    df: dataframe with cast columns
    cast_cols: list of column names (str) with the cast
    '''
    rev_cols = []
    prof_cols = []
    prof_count_cols = []
    tot_count_cols = []
    for col_name in cast_cols:
        for i in range(len(df)):
            #1. getting a dataframe with actor's movies to date only
            name = df.at[i, col_name]
            release_date = df.at[i, 'release_date']
            df_to_sum = df[(df['release_date'] < release_date) & \
                           ((df['cast_0'] == name) | (df['cast_1'] == name) | \
                            (df['cast_2'] == name) | (df['cast_3'] == name) | \
                            (df['cast_4'] == name))]
            #2. creating new cols for each actor's career rev, prof, etc. to date
            df.at[i, str(col_name)+'_rev'] = df_to_sum['revenue'].mean() #changed from .sum in prev version
            df.at[i, str(col_name)+'_prof'] = df_to_sum['profit'].mean() #changed from .sum in prev version
            df.at[i, str(col_name)+'_films'] = df_to_sum['made_money'].count()
            df.at[i, str(col_name)+'_prof_films'] = df_to_sum['made_money'].sum()
        
        #2.1 appending those new col names to a list for subsequent summing and dropping
        rev_cols.append(str(col_name)+'_rev')
        prof_cols.append(str(col_name)+'_prof')
        prof_count_cols.append(str(col_name)+'_prof_films')
        tot_count_cols.append(str(col_name)+'_films')

    #3. summing/dropping those cols so we just have cols representing top5 cast as a whole
    
    rev_to_avg = df[rev_cols].copy()
    prof_to_avg = df[prof_cols].copy()
    prof_count_to_sum = df[prof_count_cols].copy()
    tot_count_to_sum = df[tot_count_cols].copy()
    
    df['cast_rev'] = rev_to_avg.mean(axis=1) #changed from .sum in prev version
    df['cast_prof'] = prof_to_avg.mean(axis=1) #changed from .sum in prev version
    df['cast_films'] = tot_count_to_sum.sum(axis=1)
    df['cast_prof_films'] = prof_count_to_sum.sum(axis=1)
    
    df = df.drop(rev_cols, axis=1)
    df = df.drop(prof_cols, axis=1)
    df = df.drop(prof_count_cols, axis=1)
    df = df.drop(tot_count_cols, axis=1)
    
    return df

In [43]:
cast_cols = ['cast_0', 'cast_1', 'cast_2', 'cast_3', 'cast_4']

#this may take a while to run:
tdf = get_cast_metrics(df, cast_cols)  #func from cell above

In [54]:
tdf[tdf.isna().any(axis=1)];  #see rows with NaNs

In [45]:
df = tdf  #tdf used so I could experiment above/rewind if needed

In [46]:
def get_director_metrics(df, director_col):
    '''
    Iterates over column with director
    and generates new columns with the avg
    revenue, avg profit, total number of movies, and total number of
    profitable movies the director has been in
    up until the date of release for each row.
    --------
    INPUT
    df: dataframe with director columns
    director_col: column name (str) with director
    '''
    for i in range(len(df)):
        #1. getting a dataframe with director's movies to date only
        name = df.at[i, director_col]
        release_date = df.at[i, 'release_date']
        df_to_sum = df[(df['release_date'] < release_date) & (df[director_col] == name)]
        
        #2. creating new cols for each director's career avg rev, prof, etc. to date
        df.at[i, 'dir_rev'] = df_to_sum['revenue'].mean() #changed from .sum in prev version
        df.at[i, 'dir_prof'] = df_to_sum['profit'].mean() #changed from .sum in prev version
        df.at[i, 'dir_films'] = df_to_sum['made_money'].count()
        df.at[i, 'dir_prof_films'] = df_to_sum['made_money'].sum()
    
    return df

In [47]:
df = get_director_metrics(df, 'director')  #func from cell above

In [55]:
def get_writer_metrics(df, writer_col):
    '''
    Iterates over column with writer
    and generates new columns with the avg
    revenue, avg profit, total number of movies, and total number of
    profitable movies the writer has been in
    up until the date of release for each row.
    --------
    INPUT
    df: dataframe with writer columns
    writer_col: column name (str) with writer
    '''
    for i in range(len(df)):
        #1. getting a dataframe with writer's movies to date only
        name = df.at[i, writer_col]
        release_date = df.at[i, 'release_date']
        df_to_sum = df[(df['release_date'] < release_date) & (df[writer_col] == name)]
        
        #2. creating new cols for each writer's career avg rev, prof, etc. to date
        df.at[i, 'writ_rev'] = df_to_sum['revenue'].mean() #changed from .sum in prev version
        df.at[i, 'writ_prof'] = df_to_sum['profit'].mean() #changed from .sum in prev version
        df.at[i, 'writ_films'] = df_to_sum['made_money'].count()
        df.at[i, 'writ_prof_films'] = df_to_sum['made_money'].sum()
    
    return df

In [56]:
df = get_writer_metrics(df, 'writer')  #func from cell above

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799 entries, 0 to 3798
Data columns (total 29 columns):
budget             3799 non-null float64
id                 3799 non-null int64
release_date       3799 non-null datetime64[ns]
revenue            3799 non-null float64
runtime            3799 non-null float64
title              3799 non-null object
genre              3799 non-null object
profit             3799 non-null float64
made_money         3799 non-null int64
cast_0             3799 non-null object
cast_1             3799 non-null object
cast_2             3799 non-null object
cast_3             3799 non-null object
cast_4             3799 non-null object
director           3799 non-null object
writer             3799 non-null object
releases           3799 non-null int64
cast_rev           3524 non-null float64
cast_prof          3524 non-null float64
cast_films         3799 non-null float64
cast_prof_films    3799 non-null float64
dir_rev            2006 non-null float64

#### Filling the NaNs with zeros -- this isn't ideal, but the zeros do generally correspond to less prominent/experienced casts and crews, giving us additional information:

In [60]:
df = df.fillna(0)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3799 entries, 0 to 3798
Data columns (total 29 columns):
budget             3799 non-null float64
id                 3799 non-null int64
release_date       3799 non-null datetime64[ns]
revenue            3799 non-null float64
runtime            3799 non-null float64
title              3799 non-null object
genre              3799 non-null object
profit             3799 non-null float64
made_money         3799 non-null int64
cast_0             3799 non-null object
cast_1             3799 non-null object
cast_2             3799 non-null object
cast_3             3799 non-null object
cast_4             3799 non-null object
director           3799 non-null object
writer             3799 non-null object
releases           3799 non-null int64
cast_rev           3799 non-null float64
cast_prof          3799 non-null float64
cast_films         3799 non-null float64
cast_prof_films    3799 non-null float64
dir_rev            3799 non-null float64

### Getting average of cast & crew revenue, profit, etc. for each film's competitors (i.e. all other films released with +/- 2 weeks of each film):

In [62]:
def get_competitor_metrics(df, id_col):
    '''
    Iterates over each film (each row)
    and generates new columns with the average
    revenue, profit, number of movies, and number of
    profitable movies of the casts/crews from competing movies
    (i.e. those released within +/- 2 weeks) have been in.
    --------
    INPUT
    df: dataframe with cast columns
    id_col: column name (str) with film id
    '''
    for i in range(len(df)):
        #1. getting a dataframe with competing films
        id_ = df.at[i, id_col]
        release_date = df.at[i, 'release_date']
        df_to_avg = df[(df['release_date'] >= (release_date - pd.to_timedelta(14, unit='d'))) & \
               (df['release_date'] <= (release_date + pd.to_timedelta(14, unit='d'))) & \
               (df['id'] != id_)]
        
        #2. creating new cols for competing cast/crews' avg career rev, prof, etc.
        df.at[i, 'compet_cast_rev'] = df_to_avg['cast_rev'].mean()
        df.at[i, 'compet_cast_prof'] = df_to_avg['cast_prof'].mean()
        df.at[i, 'compet_cast_films'] = df_to_avg['cast_films'].mean()
        df.at[i, 'compet_cast_prof_films'] = df_to_avg['cast_prof_films'].mean()
        
        df.at[i, 'compet_dir_rev'] = df_to_avg['dir_rev'].mean()
        df.at[i, 'compet_dir_prof'] = df_to_avg['dir_prof'].mean()
        df.at[i, 'compet_dir_films'] = df_to_avg['dir_films'].mean()
        df.at[i, 'compet_dir_prof_films'] = df_to_avg['dir_prof_films'].mean()
        
        df.at[i, 'compet_writ_rev'] = df_to_avg['writ_rev'].mean()
        df.at[i, 'compet_writ_prof'] = df_to_avg['writ_prof'].mean()
        df.at[i, 'compet_writ_films'] = df_to_avg['writ_films'].mean()
        df.at[i, 'compet_writ_prof_films'] = df_to_avg['writ_prof_films'].mean()
    
    return df

In [63]:
df = get_competitor_metrics(df, 'id')

In [89]:
df = df.dropna() #dropping films without competitors in the data

In [92]:
df[df['dir_prof'] == 1082730962]

,budget,id,release_date,revenue,runtime,title,genre,profit,made_money,cast_0,cast_1,cast_2,cast_3,cast_4,director,writer,releases,cast_rev,cast_prof,cast_films,cast_prof_films,dir_rev,dir_prof,dir_films,dir_prof_films,writ_rev,writ_prof,writ_films,writ_prof_films,compet_cast_rev,compet_cast_prof,compet_cast_films,compet_cast_prof_films,compet_dir_rev,compet_dir_prof,compet_dir_films,compet_dir_prof_films,compet_writ_rev,compet_writ_prof,compet_writ_films,compet_writ_prof_films
3467,80000000.000000,324852,2017-06-15,1031552585.000000,96.000000,Despicable Me 3,Action,951552585.000000,1,Steve Carell,Kristen Wiig,Trey Parker,Miranda Cosgrove,Dana Gaier,Kyle Balda,Ken Daurio,30,131781706.260684,88278501.132479,25.000000,21.000000,1156730962.000000,1082730962.000000,1.000000,1.000000,572036645.333333,495703312.000000,3.000000,3.000000,115572227.867611,72972096.309559,29.714286,22.142857,61279580.535714,38011723.392857,1.214286,1.071429,71852422.285714,44673850.857143,0.857143,0.785714


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3600 entries, 0 to 3798
Data columns (total 41 columns):
budget                    3600 non-null float64
id                        3600 non-null int64
release_date              3600 non-null datetime64[ns]
revenue                   3600 non-null float64
runtime                   3600 non-null float64
title                     3600 non-null object
genre                     3600 non-null object
profit                    3600 non-null float64
made_money                3600 non-null int64
cast_0                    3600 non-null object
cast_1                    3600 non-null object
cast_2                    3600 non-null object
cast_3                    3600 non-null object
cast_4                    3600 non-null object
director                  3600 non-null object
writer                    3600 non-null object
releases                  3600 non-null int64
cast_rev                  3600 non-null float64
cast_prof                 3600 non-nu

#### Cleaned data to csv file:

In [97]:
# df.to_csv(path_or_buf='/home/ec2-user/clean_data_v1.csv')

In [98]:
df.to_csv(path_or_buf='/Users/ryanrappa/Desktop/dsi/film-profit-prediction/csv_files/clean_data_v1.csv')

### Next step: EDA

#### --- sandbox ---

In [66]:
def competitors_count(df, id_col):
    '''
    Iterates over each film (each row)
    and generates new columns with the average
    revenue, profit, number of movies, and number of
    profitable movies of the casts/crews from competing movies
    (i.e. those released within +/- 2 weeks) have been in.
    --------
    INPUT
    df: dataframe with cast columns
    id_col: column name (str) with film id
    '''
    counts = []
    for i in range(len(df)):
        #1. getting a dataframe with competing films
        id_ = df.at[i, id_col]
        release_date = df.at[i, 'release_date']
        df_to_avg = df[(df['release_date'] >= (release_date - pd.to_timedelta(14, unit='d'))) & \
               (df['release_date'] <= (release_date + pd.to_timedelta(14, unit='d'))) & \
               (df['id'] != id_)]
        counts.append(len(df_to_avg))
    return counts

In [67]:
compet_counts = competitors_count(df, 'id')

In [86]:
np.median(compet_counts)

7.0